In [19]:
%pip install -r ./requirements.txt

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [36]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
from llama_index.core import SimpleDirectoryReader

In [5]:
documents = SimpleDirectoryReader(input_files=["./pdf_files/mock_slide_deck_lat.pdf"]).load_data()


Ignoring wrong pointing object 7 0 (offset 0)
Ignoring wrong pointing object 9 0 (offset 0)
Ignoring wrong pointing object 11 0 (offset 0)


In [9]:
print(documents)

[Document(id_='b1988c91-de5e-40b5-a3a6-44f27dbf1ceb', embedding=None, metadata={'page_label': '1', 'file_name': 'mock_slide_deck_lat.pdf', 'file_path': 'pdf_files/mock_slide_deck_lat.pdf', 'file_type': 'application/pdf', 'file_size': 23708, 'creation_date': '2024-05-27', 'last_modified_date': '2024-05-27'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='Current Themes and RecommendationsPreventative CareAccess to CareBehavioral HealthChronic ConditionsPharmacyThemes•Preventative care, wellness visits and cancer screenings will lead to improvement•ER utilization and urgent care visits are low•Convenience care increased •Telehealth utilized by 30% of members•30% of members had a behavioral health conditions•35% of behavioral members are dependents•25% of 

In [8]:
from llama_index.core.node_parser import SentenceSplitter

# chunk_size of 1024 is a good default value
splitter = SentenceSplitter(chunk_size=1024)
# Create nodes from documents
nodes = splitter.get_nodes_from_documents(documents)
print(nodes)

/Users/dileepbarla/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


[TextNode(id_='122fde12-e3f8-40fb-9477-4d1bad9b6d0c', embedding=None, metadata={'page_label': '1', 'file_name': 'mock_slide_deck_lat.pdf', 'file_path': 'pdf_files/mock_slide_deck_lat.pdf', 'file_type': 'application/pdf', 'file_size': 23708, 'creation_date': '2024-05-27', 'last_modified_date': '2024-05-27'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='b1988c91-de5e-40b5-a3a6-44f27dbf1ceb', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '1', 'file_name': 'mock_slide_deck_lat.pdf', 'file_path': 'pdf_files/mock_slide_deck_lat.pdf', 'file_type': 'application/pdf', 'file_size': 23708, 'creation_date': '2024-05-27', 'last_modified_date': '2024-05-27'}, hash='fec2f171bbeab052a3147cded6c79742ea5ac

In [10]:
node_metadata = nodes[1].get_content(metadata_mode=True)
print(node_metadata)

page_label: 2
file_name: mock_slide_deck_lat.pdf
file_path: pdf_files/mock_slide_deck_lat.pdf
file_type: application/pdf
file_size: 23708
creation_date: 2024-05-27
last_modified_date: 2024-05-27

Recommendations and Future StatePreventative/ Wellness Initiative •Use HealthHiveto schedule and track preventive care appointments.•Receive personalized reminders for recommended screenings and vaccinations.•Access educational resources to understand the importance of preventive careAccess to Care •Use CareConnect to find nearby healthcare providers and services.•Compare prices and availability to make informed choices.•Access telemedicine options for remote consultations.Behavioral •Use MindWave to access guided meditations and therapy sessions.•Track mood and progress to identify patterns and growth.•Connect with a supportive community for encouragement.Pharmacy•Use SpecialtyScript to find nearby medical specialists.•Access affordable and convenient prescription services.•Receive personaliz

In [37]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

# LLM model
Settings.llm = OpenAI(model="gpt-4")
# embedding model
Settings.embed_model = OpenAIEmbedding()

In [38]:
from llama_index.core import SummaryIndex, VectorStoreIndex

# summary index
summary_index = SummaryIndex(nodes)


In [39]:
# vector store index
vector_index = VectorStoreIndex(nodes)


In [40]:
# summary query engine
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)

# vector query engine
vector_query_engine = vector_index.as_query_engine()

In [42]:
from llama_index.core.tools import QueryEngineTool


summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=(
        "Useful for summarization questions related to the recommendations."
    ),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "Useful for retrieving specific context from the the recommendations."
    ),
)

In [43]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector


query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    verbose=True
)

In [44]:
response = query_engine.query("What is the summary of the document?")
print(str(response))

Selecting query engine 0: The first choice is more relevant to the question because it pertains to summarizing recommendations, which is similar to summarizing a document..
The document discusses current themes and recommendations in healthcare, focusing on preventative care, access to care, behavioral health, chronic conditions, and pharmacy. It highlights the importance of preventative care, wellness visits, and cancer screenings, and notes that 30% of members utilize telehealth and have a behavioral health condition. It also mentions that obesity and hypertension are the top chronic conditions, and that specialty drugs account for a large part of high-cost claims. The document recommends using various platforms and apps like HealthHive, CareConnect, MindWave, and SpecialtyScript for scheduling and tracking preventive care appointments, finding healthcare providers, accessing therapy sessions, and managing medications.


In [45]:
print(len(response.source_nodes))

2


In [46]:
response = query_engine.query("What are the themes for preventive care?")
print(str(response))

Selecting query engine 1: The question is asking for specific information (themes for preventive care) from the recommendations, which aligns with choice 2..
The themes for preventive care include wellness visits, cancer screenings, and the belief that these will lead to improvement. Additionally, the use of telehealth by 30% of members and the increase in convenience care are also highlighted.
